In [25]:
import os
from glob import glob
from tqdm.auto import tqdm
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
# configuration
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/train.csv'
df = pd.read_csv(df_path)
print(df.head())

num2class = ['mask1', 'mask2', 'mask3', 'mask4', 'mask5',
             'incorrect_mask', 'normal']
class2num = {k : v for v, k in enumerate(num2class)}

       id  gender   race  age                    path
0  000001  female  Asian   45  000001_female_Asian_45
1  000002  female  Asian   52  000002_female_Asian_52
2  000004    male  Asian   54    000004_male_Asian_54
3  000005  female  Asian   58  000005_female_Asian_58
4  000006  female  Asian   59  000006_female_Asian_59


In [4]:
# hidden folders in '/opt/ml/input/data/train'
print(len(os.listdir(img_dir))) # 5400 folders, not 2700??
glob(os.path.join(img_dir, '._*'))[:5]

2700


[]

In [5]:
# hidden files in each image folders
os.listdir(glob(os.path.join(img_dir, '*'))[0])[:5]

['mask3.jpg', 'incorrect_mask.jpg', 'mask5.jpg', 'normal.jpg', 'mask4.jpg']

In [6]:
def delete_hidden(img_dir, img_ids):
    # confirm deletion
    answer = ''
    while not answer.isalpha():
        answer = input('Delete hidden?      [Y/n]')
        if answer.lower() == 'y':
            continue
        elif answer.lower() == 'n':
            print('End process')
            return
        else: answer = ''

    # delete hidden folders
    print('Deleting hidden folders...')
    if glob(os.path.join(img_dir, '._*')):
        for folder in tqdm(glob(os.path.join(img_dir, '._*'))):
            os.remove(folder)
    else: print('No hidden folder.')
    
    # delete hidden files
    print('Deleting hidden files...')
    for img_id in tqdm(img_ids):
        if glob(os.path.join(img_dir, img_id, '._*')):
            for file in glob(os.path.join(img_dir, img_id, '._*')):
                os.remove(file)
        else: continue
    
    print('Done')

In [7]:
# delete_hidden(img_dir, df.path.values)

In [8]:
def unifying_extensions(img_dir, img_ids):
    print('Unifying Extensions as .jpg')
    
    for img_id in tqdm(img_ids):
        for filename in os.listdir(os.path.join(img_dir, img_id)):
            # skip hidden files
            if filename[:2] == '._':
                continue
            
            # unifying extensions to .jpg
            original_file = os.path.join(img_dir, img_id, filename)
            if os.path.splitext(filename)[-1].lower() != '.jpg':
                img = Image.open(original_file)
                img.save(os.path.splitext(filename)[-1] + '.jpg')
                os.remove(original_file)
    print('Done')

In [14]:
# unifying_extensions(img_dir, df.path.values)

Unifying Extensions as .jpg



Done


In [11]:
# !pip install retina_face

In [ ]:
from retinaface import RetinaFace
from albumentations import *

In [146]:
def detect_face(img_dir, img_ids):
    x_padding = 10
    y_padding = 20
    for img_id in tqdm(img_ids, ncols=10):
        for name in num2class:
            img_file = os.path.join(img_dir, img_id, name+'.jpg')
            img = Image.open(img_file)
            bbox = RetinaFace.detect_faces(img_file)['face_1']['facial_area']
            x_padding = int((bbox[1]-bbox[0])*0.2) + 15
            y_padding = int((bbox[3]-bbox[2])*0.2)
            x_min = max(bbox[0]-x_padding, 0)
            y_min = max(bbox[1]-y_padding, 0)
            x_max = min(bbox[2]+x_padding, 384)
            y_max = min(bbox[3]+y_padding, 512)
            box = (x_min, y_min, x_max, y_max)
            cropped_img = img.crop(box=box)
            cropped_img.save(os.path.join(img_dir, img_id, name+'Cropped.jpg') 
#             plt.imshow(cropped_img)
#             plt.show()           

In [ ]:
# detect_face(img_dir, df.path.values)